In [1]:
import os
import json
from pprint import pprint

In [5]:
pythondatapath = "/home/ltv/data/datasets/source_code/codesearchnet/python/python/final/jsonl/train" 
javadatapath = "/home/ltv/data/datasets/source_code/codesearchnet/java/java/final/jsonl/train"

target_lang = 'python'
if target_lang == 'python':
    datapath = pythondatapath
elif target_lang == 'java':
    datapath = javadatapath

In [6]:
repos = set()
total_functions = 0

for filename in os.listdir(datapath):
    if filename.split(".")[-1] == "jsonl":
        full_path = os.path.join(datapath, filename)
        with open(full_path) as datafile:
            for line in datafile:
                data = json.loads(line)
                repos.add(data['repo'])
                total_functions += 1

In [7]:
print("Total codesearchnet functions:", total_functions)
print("Total repos:", len(repos))

Total codesearchnet functions: 412178
Total repos: 12231


In [18]:
def export_repos(repos, lang):
    with open("codesearchnet-%s-unique-repos.txt" % lang, "w") as repos_file:
        repos_list = list(repos)
        repos_list = sorted(repos_list)
        repos_file.write("owner\trepo\tlocation\treadme\tzip\n")
        for repo_n in repos_list:
            owner, name = tuple(repo_n.split("/"))
            repos_file.write("%s\t%s\t" % (owner, name))
            repos_file.write("https://github.com/%s\t" % repo_n),
            repos_file.write("https://raw.githubusercontent.com/%s/%s/master/README.md\t" % (owner, name))
            repos_file.write("https://codeload.github.com/%s/%s/zip/master\n" % (owner, name))
#             repos_file.write("https://github.com/%s\n" % repo_n)

In [19]:
export_repos(repos, target_lang)

In [ ]:
# if target_lang != 'python':
#     os.exit()

In [45]:
def join_with_packages(repos, packages):
    with open("codesearchnet-%s-joined-repos.txt" % target_lang, "w") as repos_file:
        repo_names = set(list(map(lambda x: x.split("/")[-1].lower(), repos)))
        
        common = []
        for package in packages:
            if package.lower() in repo_names:
                common.append(package)
                
        for repo_n in common:
            repos_file.write("%s\n" % repo_n)

with open("%s-data/%s-source-graph-packages.txt" % (target_lang, target_lang)) as sgp:
    packages = sgp.read().strip().split("\n")
    join_with_packages(repos, packages)

In [50]:
with open("codesearchnet-%s-joined-repos.txt" % target_lang, "r") as joined_repos_file:
    packages = set(joined_repos_file.read().strip().split("\n"))
    
functions = []

for filename in os.listdir(datapath):
    if filename.split(".")[-1] == "jsonl":
        full_path = os.path.join(datapath, filename)
        with open(full_path) as datafile:
            for line in datafile:
                data = json.loads(line)
                
                repo_name = data['repo'].split("/")[-1].lower()
                
                if repo_name in packages:
                    entry = {
                        'repo': data['repo'],
                        'path': data['path'],
                        'func_name': data['func_name'],
                    }
#                     entry = data['path'].replace('.py', '.').replace("/",'.') + data['func_name']
                    functions.append(entry)
                    
with open("codesearchnet-%s-joined-functions.txt" % target_lang, "w") as func_sink:
    func_sink.write("%s\n" % json.dumps(functions))
#     for f in functions:
#         func_sink.write("%s\n" % f)


In [51]:
print("Common functions:", len(functions))

Common functions: 5879
